In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pyproj import CRS
import pathlib
from pathlib import Path
from shapely import wkt
from tqdm import tqdm

import math
import codecs

from shapely import wkt

import folium
from folium import features
from folium import plugins

import gzip
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
import xml.etree.ElementTree as ET

# to read the excel 
from openpyxl import load_workbook
from openpyxl import Workbook

# import folium
from shapely.geometry import LineString, MultiLineString
import branca.colormap as cmp
from folium.plugins import Search

from tqdm import tqdm
import time

import datetime
from datetime import timedelta

# set the working directory
BASE_DIR = Path.cwd()


In [4]:
BASE_DIR

WindowsPath('E:/Github/beam/output/OutputVisualization')

In [2]:
# save as geojson
def get_foldercreation_inf():
    fname = pathlib.Path("../SF_all_trips/sf-tscore-all-trips-20PCsample-updatedRideHailFleet-updatedParking__etg/ITERS/it.4/4.linkstats.csv.gz")
    assert fname.exists(), f'No such file: {fname}'  # check that the file exists
    ctime = datetime.datetime.fromtimestamp(fname.stat().st_ctime)
    return ctime
#     return ctime.strftime("%Y-%m-%d")

def get_dataframe(_time):
    # linkstats file
    linkstats = pd.read_csv("../SF_all_trips/sf-tscore-all-trips-20PCsample-updatedRideHailFleet-updatedParking__etg/ITERS/it.4/4.linkstats.csv.gz", compression="gzip", low_memory=False)
    time = int(_time)
    linkstats = linkstats[linkstats["hour"]==(time)].copy()
    linkstats=linkstats.add_prefix("linkstats_")
    linkstats.rename(columns={('linkstats_link'): 'id'}, inplace=True)
    linkstats["id"] = linkstats["id"].astype('string')
    date_time = get_foldercreation_inf()
    if int(_time)<24:
        date_time = date_time.strftime("%Y-%m-%d")
        time_stamp = f'{int(_time):02d}'
        linkstats["date_time"] = (date_time + " " + "{}:00:00".format(f'{int(_time):02d}'))
    else:
        date_time = get_foldercreation_inf() + datetime.timedelta(days=1)
        date_time = date_time.strftime("%Y-%m-%d")
        new_time = int(_time) - 24
        linkstats["date_time"] = (date_time + " " + "{}:00:00".format(f'{abs(int(new_time)):02d}'))       
   
    return linkstats


# read the road network
sf_roadnetwork = gpd.read_file(BASE_DIR.parent.joinpath( 'Network',"sfNetwork.geojsonl"))
sf_roadnetwork = sf_roadnetwork[["id","modes","length","lanes","from","to","capacity","geometry"]]
sftimevariantnetwork =pd.DataFrame()


for time_hour in tqdm(range(0,30)):
    # get the hour and filter the linkstat file
    linkstats = get_dataframe(str(time_hour))
    # merge with featureclass of SF data
    comparision_network = sf_roadnetwork.merge(linkstats,on="id").copy()
    
    # calculate the freespeed (mph), congested speed (mph), ratio (congestedsped/freespeed)
    # linkstats
    comparision_network["linkstats_freespeed_mph"] = comparision_network["linkstats_freespeed"]*2.23694
    comparision_network["linkstats_congspd_mph"] = (comparision_network["linkstats_length"]/comparision_network["linkstats_traveltime"])*2.23694
    comparision_network["linkstats_ratio"] = comparision_network["linkstats_congspd_mph"] / comparision_network["linkstats_freespeed_mph"]   
    comparision_network["linkstats_vc_ratio"] = comparision_network["linkstats_volume"]*5 / comparision_network["capacity"]
    
    if int(time_hour)==0:
        sftimevariantnetwork = comparision_network.copy()
    else:
        sftimevariantnetwork = pd.concat([sftimevariantnetwork,comparision_network], ignore_index=True)

# lastly, export the network
# sftimevariantnetwork.to_file(BASE_DIR.parent.joinpath("exported", ("sf_timevariantnetwork.geojson")), driver='GeoJSON')

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [03:50<00:00,  7.67s/it]


In [3]:
linkstats.head()

,id,linkstats_from,linkstats_to,linkstats_hour,linkstats_length,linkstats_freespeed,linkstats_capacity,linkstats_stat,linkstats_volume,linkstats_traveltime,date_time
29,3640,961,1022,29.0,85.415,7.45,350.0,AVG,0.0,11.465101,2021-11-02 05:00:00
59,3638,961,1017,29.0,85.156,7.45,350.0,AVG,0.0,11.430336,2021-11-02 05:00:00
89,3639,1017,961,29.0,85.156,7.45,350.0,AVG,0.0,11.430336,2021-11-02 05:00:00
119,3630,1022,959,29.0,86.205,7.45,350.0,AVG,1.0,11.571141,2021-11-02 05:00:00
149,3631,959,1022,29.0,86.205,7.45,350.0,AVG,0.0,11.571141,2021-11-02 05:00:00


In [3]:
# sftimevariantnetwork.to_csv(BASE_DIR.parent.joinpath("exported", ("sf_timevariantnetwork.csv")))

In [4]:
# read the road network, incase it is already saved in the geojson file
# sftimevariantnetwork = gpd.read_file(BASE_DIR.parent.joinpath("exported", ("sf_timevariantnetwork.geojson")))

# keep only selected columns fields
sf_timevariantnetwork = sftimevariantnetwork[["id", "modes","length","lanes","capacity","geometry",
                                              'linkstats_freespeed','linkstats_volume', 'linkstats_traveltime', 
                                              'date_time', 'linkstats_freespeed_mph', 'linkstats_congspd_mph', 'linkstats_ratio', "linkstats_vc_ratio"]]
sf_timevariantnetwork['date_time']=pd.to_datetime(sf_timevariantnetwork['date_time']).dt.strftime('%Y-%m-%dT%H:%M:%S')
sf_timevariantnetwork["time"] = pd.to_datetime(sf_timevariantnetwork["date_time"]).dt.strftime('%Y-%m-%dT%H:%M:%S')


# add more green shades for 85% --> 100%
# green_shades = ['#008000', '#198c19', '#329932', '#4ca64c', '#66b266', '#7fbf7f', '#99cc99', '#b2d8b2', '#cce5cc', '#e5f2e5']
# colors for congstd speed/freespeed ratio
color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500",'#e5f2e5','#cce5cc','#b2d8b2','#99cc99','#7fbf7f','#66b266','#4ca64c','#329932', '#198c19','#008000']
# color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500","#cce5cc","#99cc99","#66b266","#008000"]
step_pct = cmp.StepColormap(
    color_range_pct,
    vmin=0, vmax=1,
    index=[0,0.2,0.3,0.5,.6,0.7,0.80,0.85, 0.87,0.89,0.91,0.93,0.95,0.97,0.99,1.00],  #for change in the colors, not used fr linear
    caption='% Speeds Difference'    #Caption for Color scale or Legend
)

# colors for congstd speed/freespeed ratio
color_range_pct_vc = ['#008000',  '#329932', '#66b266', '#99cc99', '#cce5cc', '#e5f2e5', # green shade
                      '#ffa500', "#ffb732",'#ffc966', '#ffdb99', "#ffedcc", # orange shade
                     '#ffe5e5', '#ffcccc','#ffb2b2','#ff9999','#ff6666', '#ff3232', '#ff0000' ] # red shade
# color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500","#cce5cc","#99cc99","#66b266","#008000"]
step_pct_vc = cmp.StepColormap(
    color_range_pct_vc,
    vmin=0, vmax=1,
    index=[0,0.1,0.2,0.3,0.4,0.5,
           0.55,0.6,0.65,0.7,0.75,
           0.80,0.85,0.90,0.95,0.97,0.99,1.00],  #for change in the colors, not used fr linear
    caption='Volume-to-Capacity ratio'    #Caption for Color scale or Legend
)

# colors for congested speed and freespeed 
color_range = ["#ff0000","#ff6666","#ffb2b2","#ffa500","#ffc966","#ffdb99", "#cce5cc","#99cc99","#66b266","#008000"]
step = cmp.StepColormap(color_range,vmin=0, vmax=100,index=[0,5,10,15,25,35,45,55,65,100],  #for change in the colors, not used fr linear
                        caption=' Speeds (mph)'    #Caption for Color scale or Legend
                       )


def getColorMap_pct(x):
    return str(step_pct(x))

def getColorMap_pct_vc(x):
    return str(step_pct_vc(x))

def getColorMap(x):
    return str(step(x))

sf_timevariantnetwork["fillColor_ratio"] = sf_timevariantnetwork["linkstats_ratio"].apply(getColorMap_pct)
sf_timevariantnetwork["fillColor_vc_ratio"] = sf_timevariantnetwork["linkstats_vc_ratio"].apply(getColorMap_pct_vc)
sf_timevariantnetwork["fillColor_freespeed_mph"] = sf_timevariantnetwork["linkstats_freespeed_mph"].apply(getColorMap)
sf_timevariantnetwork["fillColor_congspd_mph"] = sf_timevariantnetwork["linkstats_congspd_mph"].apply(getColorMap)

C:\Users\Transportlab\anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [5]:
def coords(geom):
    return list(geom.coords)

sf_timevariantnetwork['points'] = sf_timevariantnetwork.apply(lambda row: coords(row.geometry), axis=1)

# groupby and aggreage columns by segment_links
df1 = sf_timevariantnetwork.groupby('id').agg({'modes':'first',
                                         'length':'first',
                                         'lanes':list,
                                         'capacity':list,
                                         'geometry':'first',
                                         'linkstats_freespeed':list,
                                         'linkstats_volume':list,
                                         'linkstats_traveltime':list,
                                         'date_time':list,
                                         'linkstats_freespeed_mph':list,
                                         'linkstats_congspd_mph':list,
                                         'linkstats_ratio':list,
                                         'linkstats_vc_ratio':list,
                                         'time':list,
                                         'fillColor_ratio':list,
                                         'linkstats_volume':list,
                                         'linkstats_traveltime':list,
                                         'fillColor_freespeed_mph':list,
                                         'fillColor_congspd_mph':list,
                                         'fillColor_vc_ratio':list,
                                         'points':'first'}).reset_index()

In [13]:
# Create timemap for ratio_congestedspeed_freespeed

def coords(geom):
    return list(geom.coords)

features_ratio = [
    {
        'type':'Feature',
        "geometry":{
            'type': 'LineString',
            'coordinates': coords(d.geometry),
        },
        'properties': {
            'times': d['time'],
            'color': "black",
            'colors':d["fillColor_ratio"],
            "weight":0.6,
            "fillOpacity": 0.4,
        }
    }
    for _,d in df1.iterrows()
]

from jinja2 import Template
_template = Template("""
    {% macro script(this, kwargs) %}   
    
        L.Control.TimeDimensionCustom = L.Control.TimeDimension.extend({
            _getDisplayDateFormat: function(date){
                var newdate = new moment(date);
                console.log(newdate)
                return newdate.format("{{this.date_options}}");
            }
        });
        {{this._parent.get_name()}}.timeDimension = L.timeDimension(
            {
                period: {{ this.period|tojson }},
            }
        );
        var timeDimensionControl = new L.Control.TimeDimensionCustom(
            {{ this.options|tojson }}
        );
        {{this._parent.get_name()}}.addControl(this.timeDimensionControl);
        var geoJsonLayer = L.geoJson({{this.data}}, {
                pointToLayer: function (feature, latLng) {
                    if (feature.properties.icon == 'marker') {
                        if(feature.properties.iconstyle){
                            return new L.Marker(latLng, {
                                icon: L.icon(feature.properties.iconstyle)});
                        }
                        //else
                        return new L.Marker(latLng);
                    }
                    if (feature.properties.icon == 'circle') {
                        if (feature.properties.iconstyle) {
                            return new L.circleMarker(latLng, feature.properties.iconstyle)
                            };
                        //else
                        return new L.circleMarker(latLng);
                    }
                    //else
                    return new L.Marker(latLng);
                },
                style: function(feature) {
                    lastIdx=feature.properties.colors.length-1
                    currIdx=feature.properties.colors.indexOf(feature.properties.color);
                    if(currIdx==lastIdx){
                        feature.properties.color = feature.properties.colors[currIdx+1] 
                    }
                    else{
                        feature.properties.color =feature.properties.colors[currIdx+1] 
                    }
                    return {color: feature.properties.color}
                },
                onEachFeature: function(feature, layer) {
                    if (feature.properties.popup) {
                    layer.bindPopup(feature.properties.popup);
                    }
                }
            })
        var {{this.get_name()}} = L.timeDimension.layer.geoJson(
            geoJsonLayer,
            {
                updateTimeDimension: true,
                addlastPoint: {{ this.add_last_point|tojson }},
                duration: {{ this.duration }},
            }
        ).addTo({{this._parent.get_name()}});
    {% endmacro %}
    """)


import folium
from folium.plugins import TimestampedGeoJson

m = folium.Map(location=[37.760015, -122.447110], zoom_start=13, tiles="cartodbpositron")

t=TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features_ratio,
}, transition_time=1500,loop=True,period='PT1H', add_last_point=False,auto_play=True)
t._template=_template
t.add_to(m)
step_pct.add_to(m)

# Add title
map_title = "Ratio between Congested Speed (mph) and Free Speed (mph)"
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(map_title) 
m.get_root().html.add_child(folium.Element(title_html))

file_name = BASE_DIR.parent.joinpath("exported", ("linkstat_ratio_timemap.html"))
m.save(str(file_name))
# m

In [14]:
# Create timemap for v/c ratio

def coords(geom):
    return list(geom.coords)

features_ratio = [
    {
        'type':'Feature',
        "geometry":{
            'type': 'LineString',
            'coordinates': coords(d.geometry),
        },
        'properties': {
            'times': d['time'],
            'color': "black",
            'colors':d["fillColor_vc_ratio"],
            "weight":0.6,
            "fillOpacity": 0.4,
        }
    }
    for _,d in df1.iterrows()
]

from jinja2 import Template
_template = Template("""
    {% macro script(this, kwargs) %}   
    
        L.Control.TimeDimensionCustom = L.Control.TimeDimension.extend({
            _getDisplayDateFormat: function(date){
                var newdate = new moment(date);
                console.log(newdate)
                return newdate.format("{{this.date_options}}");
            }
        });
        {{this._parent.get_name()}}.timeDimension = L.timeDimension(
            {
                period: {{ this.period|tojson }},
            }
        );
        var timeDimensionControl = new L.Control.TimeDimensionCustom(
            {{ this.options|tojson }}
        );
        {{this._parent.get_name()}}.addControl(this.timeDimensionControl);
        var geoJsonLayer = L.geoJson({{this.data}}, {
                pointToLayer: function (feature, latLng) {
                    if (feature.properties.icon == 'marker') {
                        if(feature.properties.iconstyle){
                            return new L.Marker(latLng, {
                                icon: L.icon(feature.properties.iconstyle)});
                        }
                        //else
                        return new L.Marker(latLng);
                    }
                    if (feature.properties.icon == 'circle') {
                        if (feature.properties.iconstyle) {
                            return new L.circleMarker(latLng, feature.properties.iconstyle)
                            };
                        //else
                        return new L.circleMarker(latLng);
                    }
                    //else
                    return new L.Marker(latLng);
                },
                style: function(feature) {
                    lastIdx=feature.properties.colors.length-1
                    currIdx=feature.properties.colors.indexOf(feature.properties.color);
                    if(currIdx==lastIdx){
                        feature.properties.color = feature.properties.colors[currIdx+1] 
                    }
                    else{
                        feature.properties.color =feature.properties.colors[currIdx+1] 
                    }
                    return {color: feature.properties.color}
                },
                onEachFeature: function(feature, layer) {
                    if (feature.properties.popup) {
                    layer.bindPopup(feature.properties.popup);
                    }
                }
            })
        var {{this.get_name()}} = L.timeDimension.layer.geoJson(
            geoJsonLayer,
            {
                updateTimeDimension: true,
                addlastPoint: {{ this.add_last_point|tojson }},
                duration: {{ this.duration }},
            }
        ).addTo({{this._parent.get_name()}});
    {% endmacro %}
    """)


import folium
from folium.plugins import TimestampedGeoJson

m = folium.Map(location=[37.760015, -122.447110], zoom_start=13, tiles="cartodbpositron")

t=TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features_ratio,
}, transition_time=1500,loop=True,period='PT1H', add_last_point=False,auto_play=True)
t._template=_template
t.add_to(m)
step_pct_vc.add_to(m)

# Add title
map_title = "Volume-to-Capacity Ratio"
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(map_title) 
m.get_root().html.add_child(folium.Element(title_html))

file_name = BASE_DIR.parent.joinpath("exported","extended_run",("linkst_vc_ratio_timemap.html"))
m.save(str(file_name))
# m

In [15]:
# Create timemap for freespeed (mph)

def coords(geom):
    return list(geom.coords)

features_freespeed = [
    {
        'type':'Feature',
        "geometry":{
            'type': 'LineString',
            'coordinates': coords(d.geometry),
        },
        'properties': {
            'times': d['time'],
            'color': "black",
            'colors':d["fillColor_freespeed_mph"],
            'weight':0.6,
            "fillOpacity": 0.4, 
        }
    }
    for _,d in df1.iterrows()
]

from jinja2 import Template
_template = Template("""
    {% macro script(this, kwargs) %}
        L.Control.TimeDimensionCustom = L.Control.TimeDimension.extend({
            _getDisplayDateFormat: function(date){
                var newdate = new moment(date);
                console.log(newdate)
                return newdate.format("{{this.date_options}}");
            }
        });
        {{this._parent.get_name()}}.timeDimension = L.timeDimension(
            {
                period: {{ this.period|tojson }},
            }
        );
        var timeDimensionControl = new L.Control.TimeDimensionCustom(
            {{ this.options|tojson }}
        );
        {{this._parent.get_name()}}.addControl(this.timeDimensionControl);
        var geoJsonLayer = L.geoJson({{this.data}}, {
                pointToLayer: function (feature, latLng) {
                    if (feature.properties.icon == 'marker') {
                        if(feature.properties.iconstyle){
                            return new L.Marker(latLng, {
                                icon: L.icon(feature.properties.iconstyle)});
                        }
                        //else
                        return new L.Marker(latLng);
                    }
                    if (feature.properties.icon == 'circle') {
                        if (feature.properties.iconstyle) {
                            return new L.circleMarker(latLng, feature.properties.iconstyle)
                            };
                        //else
                        return new L.circleMarker(latLng);
                    }
                    //else
                    return new L.Marker(latLng);
                },
                style: function(feature) {
                    lastIdx=feature.properties.colors.length-1
                    currIdx=feature.properties.colors.indexOf(feature.properties.color);
                    if(currIdx==lastIdx){
                        feature.properties.color = feature.properties.colors[currIdx+1] 
                    }
                    else{
                        feature.properties.color =feature.properties.colors[currIdx+1] 
                    }
                    return {color: feature.properties.color}
                },
                onEachFeature: function(feature, layer) {
                    if (feature.properties.popup) {
                    layer.bindPopup(feature.properties.popup);
                    }
                }
            })
        var {{this.get_name()}} = L.timeDimension.layer.geoJson(
            geoJsonLayer,
            {
                updateTimeDimension: true,
                addlastPoint: {{ this.add_last_point|tojson }},
                duration: {{ this.duration }},
            }
        ).addTo({{this._parent.get_name()}});
    {% endmacro %}
    """)


import folium
from folium.plugins import TimestampedGeoJson

m = folium.Map(location=[37.760015, -122.447110], zoom_start=13, tiles="cartodbpositron")
# Add title
map_title = "Free Speed (mph)"
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(map_title) 
m.get_root().html.add_child(folium.Element(title_html))

t=TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features_freespeed,
}, transition_time=1500,loop=True,period='PT1H', add_last_point=False,auto_play=True)
t._template=_template
t.add_to(m)
step.add_to(m)

file_name = BASE_DIR.parent.joinpath("exported", ("linkstat_freespeed_timemap.html"))
m.save(str(file_name))
# m

In [16]:
# Create timemap for congested speed (mph)

def coords(geom):
    return list(geom.coords)

features_congestedspeed = [
    {
        'type':'Feature',
        "geometry":{
            'type': 'LineString',
            'coordinates': coords(d.geometry),
        },
        'properties': {
            'times': d['time'],
            'color': "black",
            'colors':d["fillColor_congspd_mph"],
            'weight':0.6,
            "fillOpacity": 0.4,
            
        }
    }
    for _,d in df1.iterrows()
]

from jinja2 import Template
_template = Template("""
    {% macro script(this, kwargs) %}
        L.Control.TimeDimensionCustom = L.Control.TimeDimension.extend({
            _getDisplayDateFormat: function(date){
                var newdate = new moment(date);
                console.log(newdate)
                return newdate.format("{{this.date_options}}");
            }
        });
        {{this._parent.get_name()}}.timeDimension = L.timeDimension(
            {
                period: {{ this.period|tojson }},
            }
        );
        var timeDimensionControl = new L.Control.TimeDimensionCustom(
            {{ this.options|tojson }}
        );
        {{this._parent.get_name()}}.addControl(this.timeDimensionControl);
        var geoJsonLayer = L.geoJson({{this.data}}, {
                pointToLayer: function (feature, latLng) {
                    if (feature.properties.icon == 'marker') {
                        if(feature.properties.iconstyle){
                            return new L.Marker(latLng, {
                                icon: L.icon(feature.properties.iconstyle)});
                        }
                        //else
                        return new L.Marker(latLng);
                    }
                    if (feature.properties.icon == 'circle') {
                        if (feature.properties.iconstyle) {
                            return new L.circleMarker(latLng, feature.properties.iconstyle)
                            };
                        //else
                        return new L.circleMarker(latLng);
                    }
                    //else
                    return new L.Marker(latLng);
                },
                style: function(feature) {
                    lastIdx=feature.properties.colors.length-1
                    currIdx=feature.properties.colors.indexOf(feature.properties.color);
                    if(currIdx==lastIdx){
                        feature.properties.color = feature.properties.colors[currIdx+1] 
                    }
                    else{
                        feature.properties.color =feature.properties.colors[currIdx+1] 
                    }
                    return {color: feature.properties.color}
                },
                onEachFeature: function(feature, layer) {
                    if (feature.properties.popup) {
                    layer.bindPopup(feature.properties.popup);
                    }
                }
            })
        var {{this.get_name()}} = L.timeDimension.layer.geoJson(
            geoJsonLayer,
            {
                updateTimeDimension: true,
                addlastPoint: {{ this.add_last_point|tojson }},
                duration: {{ this.duration }},
            }
        ).addTo({{this._parent.get_name()}});
    {% endmacro %}
    """)


import folium
from folium.plugins import TimestampedGeoJson

m = folium.Map(location=[37.760015, -122.447110], zoom_start=13, tiles="cartodbpositron")


t=TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features_congestedspeed,
}, transition_time=1500,loop=True,period='PT1H', add_last_point=False,auto_play=True)
t._template=_template
t.add_to(m)
step.add_to(m)

# Add title
map_title = "Congested Speed (mph)"
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(map_title) 
m.get_root().html.add_child(folium.Element(title_html))


file_name = BASE_DIR.parent.joinpath("exported", ("linkstat_congestedspeed_timemap.html"))
m.save(str(file_name))
# m

In [9]:
# get map for each different time

In [7]:
# static maps for congestedspeed/freespeed

def get_dataframe(_time):
    # linkstats file
    linkstats = pd.read_csv("../SF_all_trips/sf-tscore-all-trips-20PCsample-updatedRideHailFleet-updatedParking__etg/ITERS/it.4/4.linkstats.csv.gz", compression="gzip", low_memory=False)
#     unmodified_linkstats = pd.read_csv(BASE_DIR.parent.joinpath("runs", "sf-tscore-int-int-trips-model-network-events-20PC-sample-bpr-func__tlm","ITERS","it.30", "30.linkstats_unmodified.csv.gz"),compression="gzip", low_memory=False)
    time = int(_time)
    linkstats = linkstats[linkstats["hour"]==(time)].copy()
    linkstats=linkstats.add_prefix("linkstats_")
    linkstats.rename(columns={('linkstats_link'): 'id'}, inplace=True)
    linkstats["id"] = linkstats["id"].astype('string')
   
    return linkstats

def highlight_function(feature):
    return {"fillColor": "#ffff00", "color": "#ffff00", "weight": 5,"fillOpacity": 0.40 }


color_range_pct = ["#ff0000","#ff6666","#ffb2b2","#ffdb99","#ffc966", "#ffa500",'#e5f2e5','#cce5cc','#b2d8b2','#99cc99','#7fbf7f','#66b266','#4ca64c','#329932', '#198c19','#008000']

step_pct = cmp.StepColormap(
    color_range_pct,
    vmin=0, vmax=1,
    index=[0,0.2,0.3,0.5,.6,0.7,0.80,0.85, 0.87,0.89,0.91,0.93,0.95,0.97,0.99,1.00],  #for change in the colors, not used fr linear
    caption='% Speeds Difference'    #Caption for Color scale or Legend
)

# read the road network
sf_roadnetwork = gpd.read_file(BASE_DIR.parent.joinpath("Network", "sfNetwork.geojsonl"))
sf_roadnetwork = sf_roadnetwork[["id","modes","length","lanes","from","to","capacity","geometry"]]

for time_hour in tqdm(range(0,30)):
    # set the map
    pct_m = folium.Map([37.760015, -122.447110], zoom_start=13, tiles="cartodbpositron")
    # get the hour and filter the linkstat file
    linkstats = get_dataframe(str(time_hour))
    # merge with featureclass of SF data
    comparision_network = sf_roadnetwork.merge(linkstats,on="id")
    
    # calculate the freespeed (mph), congested speed (mph), ratio (congestedsped/freespeed)
    # linkstats
    comparision_network["linkstats_freespeed_mph"] = comparision_network["linkstats_freespeed"]*2.23694
    comparision_network["linkstats_congspd_mph"] = (comparision_network["linkstats_length"]/comparision_network["linkstats_traveltime"])*2.23694
    comparision_network["linkstats_ratio"] = comparision_network["linkstats_congspd_mph"] / comparision_network["linkstats_freespeed_mph"]
    
    time_stamp = ""
    # folium
    if time_hour<30:        
        time_stamp = f'{time_hour:02d}'
        layer_name = str(time_stamp)        
    
        
#     layer_name=str(str(time_hour) + (' am' if time_hour < 12 else ' pm'))
    ratio_feature_group = folium.FeatureGroup(name=layer_name)
    
    pct_feature_group = folium.GeoJson(comparision_network, 
                                   name = ("Hour - " + layer_name),
                                   style_function=lambda x: {
                                       "fillColor": step_pct(x["properties"]["linkstats_ratio"]),
                                       "color": step_pct(x["properties"]["linkstats_ratio"]),
                                       "fillOpacity": 0.2,
                                       "weight":1,
                                   },
                                   tooltip=folium.GeoJsonTooltip(fields=["id","length", "linkstats_freespeed_mph", "linkstats_traveltime","linkstats_congspd_mph"], 
                                                                 aliases=["Link ID", "Segment Length (m)", "Freespeed (mph)", "Travel time (sec)", "Congested Speed (mph)"], localize=True),
                                   popup = folium.GeoJsonPopup(fields=["id","length", "linkstats_freespeed_mph", "linkstats_traveltime","linkstats_congspd_mph"], 
                                                                 aliases=["Link ID", "Segment Length (m)", "Freespeed (mph)", "Travel time (sec)", "Congested Speed (mph)"], localize=True),
                                   highlight_function=highlight_function,
                                   zoom_on_click=True
                                  ).add_to(ratio_feature_group)
    # Add search functionality to the map
    search_link = Search(layer=pct_feature_group, geom_type="LineString", placeholders = "Search for Link ID", 
                     collapsed="False", search_label = 'id', search_zoom = 17, position='topleft',
                     ).add_to(pct_m)
    
    ratio_feature_group.add_to(pct_m)
    folium.LayerControl().add_to(pct_m)
    
    map_title = "Ratio between Congested Speed and Free Speed"
    title_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format(map_title)
    pct_m.get_root().html.add_child(folium.Element(title_html))
    # save the file
    file_name = BASE_DIR.parent.joinpath("exported", ("linkstat_ratio_timemap_{}.html").format(time_stamp))
    pct_m.save(str(file_name))

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [32:58<00:00, 65.95s/it]
